In [ ]:
from hashlib import sha1
import hmac
import binascii
def getUrl(request):
    devId = 3002228
    key = '5bef2099-6493-48e4-b344-e37e99530ce6'
    request = request + ('&' if ('?' in request) else '?')
    raw = request+'devid={0}'.format(devId)
    #print(raw.encode())
    key_bin = key.encode()
    raw_bin = raw.encode() 
    hashed = hmac.new(key_bin, raw_bin, sha1)
    signature = hashed.hexdigest()
    return 'http://timetableapi.ptv.vic.gov.au'+raw+'&signature={1}'.format(devId, signature)
#print(getUrl('/v3/route_types'))


import requests
import json
def download_get(url_name,file_path):
    url_str = getUrl(url_name)
    print(url_str)
    response = requests.get(url= url_str)
    print(response.status_code)
    print(response.text)
    with open(file_path,'w') as f:
        f.write(response.text)
def download_get_to_dict(url_name):
    url_str = getUrl(url_name)
    print(url_str)
    response = requests.get(url= url_str)
    print(response.status_code)
    print(response.text)
    return json.loads(response.text)

#download_get('/v3/route_types','/Users/chunhua/tmp_share/heyonglin/route_types.json')
#download_get('/v3/routes','/Users/heyonglin/routes.json')

routes_json_file = '../data/routes.json'
# routes_json_file = '../data/routes.json'
with open(routes_json_file) as f:
    route_dict = json.load(f)
    # print(route_dict.keys())
    stops_files = [open(f'../data/stops_file_{i}.json','a')  for i in range(5)]
    for stop_file in stops_files:
        stop_file.write('[\n')
    is_first_line = [True,True,True,True,True]
    for route_item in route_dict['routes']:
        route_id = route_item['route_id']
        route_type = route_item['route_type']
        url_name = f'/v3/stops/route/{route_id}/route_type/{route_type}'
        res = download_get_to_dict(url_name)
        stops = res['stops']
        for stop in stops:
            if not is_first_line[route_type]:
                stops_files[route_type].write(',\n')
            json.dump(stop,stops_files[route_type])
            is_first_line[route_type] = False
    
    for stop_file in stops_files:
        stop_file.write('\n]')
        stop_file.close()  

In [188]:
def to_lower(text):
    text = text.lower()
    return text

In [195]:
import pandas as pd
train_stops_file = pd.read_json('../data/stops_file_0.json')
train_stops_file = train_stops_file[['stop_suburb','stop_latitude','stop_longitude','stop_id','stop_name']]
train_stops_file['stop_suburb'] = train_stops_file['stop_suburb'].apply(to_lower)
train_stops_file['stop_name'] = train_stops_file['stop_name'].drop_duplicates()

In [196]:
train_stops_file = train_stops_file.dropna()
train_stops_file

,stop_suburb,stop_latitude,stop_longitude,stop_id,stop_name
0,ashburton,-37.868317,145.079666,1002,Alamein Station
1,ashburton,-37.861970,145.081345,1010,Ashburton Station
2,hawthorn east,-37.822400,145.045837,1012,Auburn Station
3,burnley,-37.827557,145.007553,1030,Burnley Station
4,glen iris,-37.851562,145.080500,1031,Burwood Station
...,...,...,...,...,...
347,williams landing,-37.869870,144.747452,1225,Williams Landing Station
348,yarraville,-37.815850,144.889938,1216,Yarraville Station
355,williamstown,-37.857330,144.889069,1146,North Williamstown Station
361,williamstown,-37.867756,144.905319,1211,Williamstown Station


In [197]:
def change_name(text):
    if(text == 'melbourne city'):
        return 'melbourne'
    else:
        return text
train_stops_file['stop_suburb'] = train_stops_file['stop_suburb'].apply(change_name)

In [364]:
tram_stops_file = pd.read_json('../data/stops_file_1.json')
tram_stops_file = tram_stops_file[['stop_suburb','stop_latitude','stop_longitude','stop_id','stop_name']]
tram_stops_file['stop_suburb'] = tram_stops_file['stop_suburb'].apply(to_lower)
tram_stops_file['stop_name'] = tram_stops_file['stop_name'].drop_duplicates()
tram_stops_file['stop_suburb'] = tram_stops_file['stop_suburb'].apply(change_name)
tram_stops_file = tram_stops_file.dropna()
tram_stops_file

,stop_suburb,stop_latitude,stop_longitude,stop_id,stop_name
0,brunswick east,-37.769787,144.971954,2227,Albert St/Lygon St #123
1,brunswick east,-37.761890,144.974060,2240,Albion St/Holmes St #127
2,brunswick east,-37.762370,144.973267,2231,Albion St/Lygon St #127
3,southbank,-37.821842,144.969513,2203,Arts Precinct/St Kilda Rd #14
4,southbank,-37.823814,144.967255,2266,Arts Precinct/Sturt St #17
...,...,...,...,...,...
1509,prahran,-37.850563,144.984039,2978,Punt Rd/High St #28
1516,coburg,-37.755560,144.964279,2236,Sydney Rd/Moreland Rd #132
1517,prahran,-37.852440,144.998100,2640,The Avenue/High St #33
1519,glen iris,-37.858078,145.041046,2622,Tooronga Rd/High St #48


In [365]:
bus_stops_file = pd.read_json('../data/stops_file_2.json')
bus_stops_file = bus_stops_file[['stop_suburb','stop_latitude','stop_longitude','stop_id','stop_name']]
bus_stops_file['stop_suburb'] = bus_stops_file['stop_suburb'].apply(to_lower)
bus_stops_file['stop_name'] = bus_stops_file['stop_name'].drop_duplicates()
bus_stops_file['stop_suburb'] = bus_stops_file['stop_suburb'].apply(change_name)
bus_stops_file = bus_stops_file.dropna()
bus_stops_file 

,stop_suburb,stop_latitude,stop_longitude,stop_id,stop_name
0,avondale heights,-37.762234,144.859680,23570,Avondale Heights Reserve/Doyle St
1,maribyrnong,-37.769444,144.872772,31671,Canning Reserve/Canning St
3,avondale heights,-37.769413,144.857254,25271,Canning St/Herbert St
4,maribyrnong,-37.769013,144.877823,11239,Central Park Ave/Cordite Ave
5,avondale heights,-37.759260,144.857391,25277,Davis Ave/Skewes St
...,...,...,...,...,...
29797,hampton park,-38.030907,145.258270,19099,Robert Booth Reserve/Somerville Rd
29798,hampton park,-38.042637,145.270248,17513,Robert Ct/Huntington Dr
29799,narre warren south,-38.050587,145.296158,17528,Seebeck Dr/Sneddon Dr
29802,hampton park,-38.039837,145.280609,17583,The Ridge/Ralph Cres


In [366]:
vline_stops_file = pd.read_json('../data/stops_file_3.json')
vline_stops_file = vline_stops_file[['stop_suburb','stop_latitude','stop_longitude','stop_id','stop_name']]
vline_stops_file['stop_suburb'] = vline_stops_file['stop_suburb'].apply(to_lower)
vline_stops_file['stop_name'] = vline_stops_file['stop_name'].drop_duplicates()
vline_stops_file['stop_suburb'] = vline_stops_file['stop_suburb'].apply(change_name)
vline_stops_file = vline_stops_file.dropna()
vline_stops_file 

,stop_suburb,stop_latitude,stop_longitude,stop_id,stop_name
0,ararat,-37.282440,142.936900,1501,Ararat Railway Station
1,ararat,-37.283640,142.931992,26759,Ararat Town Hall/Barkly St
2,macarthur,-38.033012,141.999741,21122,Eversley St/Hamilton - Port Fairy Rd
3,hamilton,-37.746600,142.028275,4125,Hamilton Station/Station St
4,dunkeld,-37.649464,142.342560,4130,Information Centre/Glenelg Hwy
...,...,...,...,...,...
1381,alexandra,-37.190964,145.709076,26124,Perkiins St/Grant St
1383,trawool,-37.093380,145.204620,28302,Upper Goulburn Rd/Goulburn Valley Hwy
1391,clunes,-37.303142,143.781570,1591,Clunes Railway Station
1405,talbot,-37.172900,143.705627,1594,Talbot Railway Station


In [22]:
from urllib import parse
import html2text as html2text
import requests
import json
url = "https://vahi.vic.gov.au/views/ajax?_wrapper_format=drupal_ajax"
for i in range(27):
    post_body = parse.urlencode({"view_name":"hospital_and_health_services","view_display_id":"page_1","page":i})
    response = requests.post(url= url,headers={
        "accept": "application/json, text/javascript, */*; q=0.01",
        "accept-language": "zh-CN,zh;q=0.9",
        "content-type": "application/x-www-form-urlencoded; charset=UTF-8",
        "sec-ch-ua": "\"Chromium\";v=\"104\", \" Not A;Brand\";v=\"99\", \"Google Chrome\";v=\"104\"",
        "sec-ch-ua-mobile": "?0",
        "sec-ch-ua-platform": "\"Windows\"",
        "sec-fetch-dest": "empty",
        "sec-fetch-mode": "cors",
        "sec-fetch-site": "same-origin",
        "x-requested-with": "XMLHttpRequest"
      },data = post_body)
    print(response.status_code)
    print(response.text)
    with open(f'v2_ajax_download_{i:02d}','w') as f:
        f.write(response.text)



with open('../data/v2_ajax_download_00') as f:
    d = json.load(f)
    print(len(d))
    geofield_google_map = d[0]["settings"]["geofield_google_map"]
    with open('../data/v2_ajax_download.csv','w') as csv_file:
        csv_file.write('"name","address"\n')
        for geofield in geofield_google_map:
            for feature in geofield_google_map[geofield]["data"]["features"]:
                #print(feature)
                name_with_tag = feature["properties"]["data"]["view_node"]
                name = name_with_tag[name_with_tag.index('>')+1 : name_with_tag.index('<',1)]
                address = feature["properties"]["data"]["field_address"]
                #print(html2text.html2text(),end=";")
                csv_file.write(f'"{name}","{address}"\n')

5


In [355]:
geo_df = pd.read_excel("../data/postcode_in_VIC.csv")
geo_df = geo_df[geo_df['Postcode'] > 0 ]
geo_df = geo_df[geo_df['Postcode'] < 8000 ]
geo_df['Locality'] = geo_df['Locality'].apply(to_lower)


In [371]:
import math
EARTH_REDIUS = 6378.137


def rad(d):
    return d * math.pi / 180.0


def getDistance(lat1, lng1, lat2, lng2):
    radLat1 = rad(lat1)
    radLat2 = rad(lat2)
    a = radLat1 - radLat2
    b = rad(lng1) - rad(lng2)
    s = 2 * math.asin(math.sqrt(math.pow(math.sin(a/2), 2) + math.cos(radLat1) * math.cos(radLat2) * math.pow(math.sin(b/2), 2)))
    s = s * EARTH_REDIUS
    return s


def find_postcode(geo_df,lat, lng, suburb):
    if(suburb == 'melbourne'):
        distance = 10000
        result = None
        for index, item in geo_df.iterrows():
            postcode = item['Postcode']
            lat2, lng2 = item['Latitude'], item['Longitude']
            d = getDistance(lat, lng, lat2, lng2)
            if (d < distance):
                distance = d
                result = postcode
        return int(result)
    else:
        for index, item in geo_df.iterrows():
            locality = item['Locality']
            if locality == suburb:
                return int(item['Postcode'])


In [356]:
train_stops_file['postcode'] = None
for index, item in train_stops_file.iterrows():
    lat = item['stop_latitude']
    lng = item['stop_longitude']
    stop_suburb = item['stop_suburb']
    postcode = find_postcode(geo_df, lat, lng, stop_suburb)
    train_stops_file.loc[index, 'postcode'] = postcode

melbourne
melbourne
melbourne
melbourne
melbourne


In [372]:
tram_stops_file['postcode'] = None
for index, item in tram_stops_file.iterrows():
    lat = item['stop_latitude']
    lng = item['stop_longitude']
    stop_suburb = item['stop_suburb']
    postcode = find_postcode(geo_df, lat, lng, stop_suburb)
    tram_stops_file.loc[index, 'postcode'] = postcode
    

In [373]:
bus_stops_file['postcode'] = None
for index, item in bus_stops_file.iterrows():
    lat = item['stop_latitude']
    lng = item['stop_longitude']
    stop_suburb = item['stop_suburb']
    postcode = find_postcode(geo_df, lat, lng, stop_suburb)
    bus_stops_file.loc[index, 'postcode'] = postcode

In [374]:
vline_stops_file['postcode'] = None
for index, item in vline_stops_file.iterrows():
    lat = item['stop_latitude']
    lng = item['stop_longitude']
    stop_suburb = item['stop_suburb']
    postcode = find_postcode(geo_df, lat, lng, stop_suburb)
    vline_stops_file.loc[index, 'postcode'] = postcode

In [378]:
vline_stops_file

,stop_suburb,stop_latitude,stop_longitude,stop_id,stop_name,postcode
0,ararat,-37.282440,142.936900,1501,Ararat Railway Station,3377
1,ararat,-37.283640,142.931992,26759,Ararat Town Hall/Barkly St,3377
2,macarthur,-38.033012,141.999741,21122,Eversley St/Hamilton - Port Fairy Rd,3286
3,hamilton,-37.746600,142.028275,4125,Hamilton Station/Station St,3300
4,dunkeld,-37.649464,142.342560,4130,Information Centre/Glenelg Hwy,3294
...,...,...,...,...,...,...
1381,alexandra,-37.190964,145.709076,26124,Perkiins St/Grant St,3714
1383,trawool,-37.093380,145.204620,28302,Upper Goulburn Rd/Goulburn Valley Hwy,3660
1391,clunes,-37.303142,143.781570,1591,Clunes Railway Station,3370
1405,talbot,-37.172900,143.705627,1594,Talbot Railway Station,3371


In [377]:
bus_stops_file[bus_stops_file['stop_suburb'] == 'melbourne']

,stop_suburb,stop_latitude,stop_longitude,stop_id,stop_name,postcode
3282,melbourne,-37.816044,144.951981,20084,Skybus Coach Terminal/Spencer St,3008
6413,melbourne,-37.815266,144.960700,14163,Bourke St/Queen St,3000
6433,melbourne,-37.812510,144.961792,29400,Elizabeth St/Lonsdale St,3000
6435,melbourne,-37.810440,144.968857,18051,Exhibition St/Lonsdale St,3053
6446,melbourne,-37.812633,144.960648,17816,Hardware Lane/Lonsdale St,3000
6458,melbourne,-37.813793,144.960300,18396,Little Bourke St/Queen St,3000
6459,melbourne,-37.816177,144.961100,18384,Little Collins St/Queen St,3000
6460,melbourne,-37.808840,144.968964,18049,Little Lonsdale St/Exhibition St,3053
6465,melbourne,-37.811565,144.965027,18054,Melbourne Central/Lonsdale St,3053
7847,melbourne,-37.808582,144.966309,18041,La Trobe St/Russell St,3053


In [359]:
train_stops_file.to_csv('../data/train_stops_file.csv')
train_stops_file.to_csv('../data/train_stops_file.csv')
train_stops_file.to_csv('../data/train_stops_file.csv')
train_stops_file.to_csv('../data/train_stops_file.csv')